In [5]:
%%shell
sudo apt-get update -y
sudo apt-get install -y r-base

Hit:1 http://security.ubuntu.com/ubuntu jammy-security InRelease
Hit:2 https://cli.github.com/packages stable InRelease
Hit:3 https://cloud.r-project.org/bin/linux/ubuntu jammy-cran40/ InRelease
Hit:4 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2204/x86_64  InRelease
Hit:5 https://r2u.stat.illinois.edu/ubuntu jammy InRelease
Hit:6 http://archive.ubuntu.com/ubuntu jammy InRelease
Hit:7 http://archive.ubuntu.com/ubuntu jammy-updates InRelease
Hit:8 https://ppa.launchpadcontent.net/deadsnakes/ppa/ubuntu jammy InRelease
Hit:9 https://ppa.launchpadcontent.net/graphics-drivers/ppa/ubuntu jammy InRelease
Hit:10 http://archive.ubuntu.com/ubuntu jammy-backports InRelease
Hit:11 https://ppa.launchpadcontent.net/ubuntugis/ppa/ubuntu jammy InRelease
Reading package lists... Done
W: Skipping acquire of configured file 'main/source/Sources' as repository 'https://r2u.stat.illinois.edu/ubuntu jammy InRelease' does not seem to provide it (sources.list entry misspelt?)
Reading packag

In [6]:
%%shell
pip install rpy2

In [4]:
%reload_ext rpy2.ipython

In [8]:
%%shell
mkdir -p ~/.kaggle
cp /content/sample_data/kaggle.json ~/.kaggle/
chmod 600 ~/.kaggle/kaggle.json

In [9]:
%%shell
pip install kaggle

In [10]:
%%shell

kaggle datasets download wardabilal/spotify-global-music-dataset-20092025

Dataset URL: https://www.kaggle.com/datasets/wardabilal/spotify-global-music-dataset-20092025
License(s): CC0-1.0
  0% 0.00/1.23M [00:00<?, ?B/s]
100% 1.23M/1.23M [00:00<00:00, 696MB/s]


In [11]:
%%R
system("unzip -o /content/spotify-global-music-dataset-20092025.zip -d /content/")
df <- read.csv("/content/track_data_final.csv")
head(df, 10)
summary(df)

   track_id          track_name         track_number     track_popularity
 Length:8778        Length:8778        Min.   :  1.000   Min.   :  0.00  
 Class :character   Class :character   1st Qu.:  1.000   1st Qu.: 39.00  
 Mode  :character   Mode  :character   Median :  4.000   Median : 58.00  
                                       Mean   :  5.749   Mean   : 52.23  
                                       3rd Qu.:  9.000   3rd Qu.: 71.00  
                                       Max.   :102.000   Max.   :100.00  
                                                                         
 track_duration_ms   explicit         artist_name        artist_popularity
 Min.   :     0    Length:8778        Length:8778        Min.   :  0.00   
 1st Qu.:173921    Class :character   Class :character   1st Qu.: 60.00   
 Median :207276    Mode  :character   Mode  :character   Median : 74.00   
 Mean   :210158                                          Mean   : 69.95   
 3rd Qu.:239693                  

In [9]:
%%R
# Versión final solicitada — modular y comentada
# INPUT fijo: /content/track_data_final.csv
# OUTPUT: processed_spotify_eda_final.csv + gráficas en eda_plots/

# ------------------------------
# A) Parámetros y configuración
# ------------------------------
params <- list(
  csv_path = "/content/track_data_final.csv",           # ruta fija proporcionada
  processed_path = "processed_spotify_eda_final.csv",
  out_dir = "eda_plots",
  req_pkgs = c("tidyverse", "lubridate", "janitor", "naniar", "scales", "ggcorrplot", "readr"),
  is_hit_thresholds = c(mega_hit = 90, hit = 80, popular = 60), # puedes ajustar
  year_limits = c(2000, 2025),
  max_vars_for_corr = 20, # Added: Limit for correlation plot variables
  verbose = TRUE
)

msg <- function(...) if (isTRUE(params$verbose)) message(paste0(...))

# ------------------------------
# B) Instalar / cargar paquetes
# ------------------------------
install_and_load <- function(pkgs) {
  to_install <- pkgs[!(pkgs %in% installed.packages()[, "Package"])]
  if (length(to_install) > 0) install.packages(to_install, repos = "https://cloud.r-project.org")
  invisible(lapply(pkgs, library, character.only = TRUE, warn.conflicts = FALSE, quietly = TRUE))
}
install_and_load(params$req_pkgs)
library(tidyverse); library(lubridate); library(janitor); library(naniar); library(scales); library(ggcorrplot); library(readr)

# ------------------------------
# C) Utilidades
# ------------------------------
parse_num_safe <- function(x) {
  x <- ifelse(is.na(x) | x == "", NA_character_, x)
  x_clean <- gsub("[,\\s+]", "", x)
  suppressWarnings(as.numeric(x_clean))
}
ensure_dir <- function(d) if (!dir.exists(d)) dir.create(d, recursive = TRUE)

# ------------------------------
# D) Leer CSV
# ------------------------------
msg("Leyendo CSV: ", params$csv_path)
if (!file.exists(params$csv_path)) stop("No se encontró el archivo: ", params$csv_path)
raw <- readr::read_csv(params$csv_path, col_types = cols(.default = "c"), show_col_types = FALSE)
msg("Lectura completada. Filas: ", nrow(raw), " Columnas: ", ncol(raw))

# ------------------------------
# E) Inspección inicial y limpieza de nombres
# ------------------------------
df0 <- raw %>% janitor::clean_names()
msg("Columnas detectadas: ", paste(names(df0), collapse = ", "))

# convertir columnas principales que vienen como character a numeric/logical de forma segura
numeric_expected <- c("track_number","track_popularity","track_duration_ms","artist_popularity","artist_followers","album_total_tracks")
for (cname in intersect(numeric_expected, names(df0))) df0[[cname]] <- parse_num_safe(df0[[cname]])

if ("explicit" %in% names(df0)) {
  df0 <- df0 %>% mutate(explicit = case_when(
    is.na(explicit) ~ NA,
    tolower(explicit) %in% c("true","t","1","yes","y") ~ TRUE,
    tolower(explicit) %in% c("false","f","0","no","n") ~ FALSE,
    TRUE ~ NA
  ))
}

# parseo seguro fecha album_release_date si existe
if ("album_release_date" %in% names(df0)) {
  df0 <- df0 %>% mutate(album_release_date = ifelse(album_release_date == "", NA_character_, album_release_date)) %>%
    mutate(album_release_date = lubridate::parse_date_time(album_release_date, orders = c("Ymd","ymd","Y-m-d","Y","dmy"), truncated = 3)) %>%
    mutate(album_release_date = as_date(album_release_date))
}

# ------------------------------
# F) Feature engineering (crea columnas útiles para el modelo de IA)
# ------------------------------
feature_engineer <- function(df) {
  df1 <- df

  # 1) Fechas: año, mes, día, década
  if ("album_release_date" %in% names(df1)) {
    df1 <- df1 %>% mutate(
      release_year = if_else(!is.na(album_release_date), lubridate::year(album_release_date), NA_integer_),
      release_month = if_else(!is.na(album_release_date), lubridate::month(album_release_date), NA_integer_),
      release_day = if_else(!is.na(album_release_date), lubridate::day(album_release_date), NA_integer_),
      release_decade = if_else(!is.na(release_year), (release_year %/% 10) * 10, NA_integer_)
    )
  } else {
    df1 <- df1 %>% mutate(release_year = NA_integer_, release_month = NA_integer_, release_day = NA_integer_, release_decade = NA_integer_)
  }

  # 2) Duración en minutos
  if ("track_duration_ms" %in% names(df1)) {
    df1 <- df1 %>% mutate(duration_min = as.numeric(track_duration_ms) / 60000)
  } else if ("track_duration_min" %in% names(df1)) {
    df1 <- df1 %>% mutate(duration_min = parse_num_safe(track_duration_min))
  } else df1 <- df1 %>% mutate(duration_min = NA_real_)

  # 3) Popularity numeric y bins
  if ("track_popularity" %in% names(df1)) {
    df1 <- df1 %>% mutate(track_popularity = as.numeric(track_popularity))
    t <- params$is_hit_thresholds
    df1 <- df1 %>% mutate(popularity_label = case_when(
      is.na(track_popularity) ~ NA_character_,
      track_popularity >= t["mega_hit"] ~ "mega_hit",
      track_popularity >= t["hit"] ~ "hit",
      track_popularity >= t["popular"] ~ "popular",
      TRUE ~ "regular"
    ) %>% factor(levels = c("regular","popular","hit","mega_hit")))
  } else df1 <- df1 %>% mutate(popularity_label = NA)

  # 4) Transformaciones útiles: log(artist_followers + 1), followers per popularity
  if ("artist_followers" %in% names(df1)) {
    df1 <- df1 %>% mutate(artist_followers = as.numeric(artist_followers), artist_followers_log = log1p(artist_followers))
  } else df1 <- df1 %>% mutate(artist_followers_log = NA_real_)

  # 5) Artist popularity numeric (si existe) y z-score (normalización simple)
  if ("artist_popularity" %in% names(df1)) {
    df1 <- df1 %>% mutate(artist_popularity = as.numeric(artist_popularity))
    df1 <- df1 %>% mutate(artist_popularity_z = (artist_popularity - mean(artist_popularity, na.rm = TRUE)) / sd(artist_popularity, na.rm = TRUE))
  } else df1 <- df1 %>% mutate(artist_popularity_z = NA_real_)

  # 6) Texto del título: longitud, número de palabras, flag featured/collab
  if ("track_name" %in% names(df1)) {
    df1 <- df1 %>% mutate(
      title_len = nchar(track_name),
      title_n_words = if_else(!is.na(track_name), str_count(track_name, "\\S+"), NA_integer_),
      title_has_feat = if_else(!is.na(track_name) & str_detect(tolower(track_name), "feat\\.|ft\\.|feat |featuring|\\(ft\\.|\\(feat\\."), TRUE, FALSE)
    )
  } else {
    df1 <- df1 %>% mutate(title_len = NA_integer_, title_n_words = NA_integer_, title_has_feat = NA)
  }

  # 7) Géneros: contar número de géneros, primera etiqueta, normalizar string
  if ("artist_genres" %in% names(df1)) {
    df1 <- df1 %>% mutate(artist_genres = if_else(is.na(artist_genres) | artist_genres == "", NA_character_, artist_genres))
    df1 <- df1 %>% mutate(
      n_genres = if_else(is.na(artist_genres), 0L, str_count(artist_genres, ",") + 1L),
      genre_primary = if_else(is.na(artist_genres), NA_character_, str_trim(str_split_fixed(artist_genres, ",", 2)[,1]))
    )
  } else df1 <- df1 %>% mutate(n_genres = NA_integer_, genre_primary = NA_character_)

  # 8) album features: album track ratio (track_number / album_total_tracks), album_age_years
  if (all(c("track_number","album_total_tracks") %in% names(df1))) {
    df1 <- df1 %>% mutate(album_total_tracks = as.numeric(album_total_tracks), track_number = as.numeric(track_number))
    df1 <- df1 %>% mutate(track_pos_in_album = if_else(!is.na(track_number) & !is.na(album_total_tracks) & album_total_tracks > 0, track_number / album_total_tracks, NA_real_))
  } else df1 <- df1 %>% mutate(track_pos_in_album = NA_real_)

  if ("album_release_date" %in% names(df1) && any(!is.na(df1$album_release_date))) {
    max_d <- max(df1$album_release_date, na.rm = TRUE)
    df1 <- df1 %>% mutate(album_age_years = as.numeric(difftime(max_d, album_release_date, units = "weeks")) / 52.25)
  } else df1 <- df1 %>% mutate(album_age_years = NA_real_)

  # 9) Artist top-k flag (top 20) — útil para reducir cardinalidad
  if ("artist_name" %in% names(df1)) {
    topk <- df1 %>% count(artist_name, sort = TRUE) %>% slice_max(n, n = 20) %>% pull(artist_name)
    df1 <- df1 %>% mutate(artist_top20 = if_else(artist_name %in% topk, artist_name, "other") %>% as.factor())
  } else df1 <- df1 %>% mutate(artist_top20 = NA)

  # 10) Normalizaciones simples (para inspección; no sobrescriben valores originales)
  if ("duration_min" %in% names(df1)) df1 <- df1 %>% mutate(duration_min_z = (duration_min - mean(duration_min, na.rm = TRUE)) / sd(duration_min, na.rm = TRUE))
  if ("track_popularity" %in% names(df1)) df1 <- df1 %>% mutate(track_popularity_z = (track_popularity - mean(track_popularity, na.rm = TRUE)) / sd(track_popularity, na.rm = TRUE))

  # convertir algunos logicals/factors
  df1 <- df1 %>% mutate(across(where(is.logical), ~ .x)) # mantener booleans
  df1
}

msg("Aplicando feature engineering...")
df <- feature_engineer(df0)

# Guardar DF procesado
ensure_dir(params$out_dir)
readr::write_csv(df, params$processed_path)
msg("Procesado guardado en: ", params$processed_path)

# ------------------------------
# G) Estadísticas resumidas (no guardamos archivos extra, solo console)
# ------------------------------
msg("Resumen rápido (numéricas):")
num_cols <- df %>% select(where(is.numeric)) %>% names()
if (length(num_cols) > 0) print(df %>% summarise(across(all_of(num_cols), list(mean = ~ mean(.x, na.rm=TRUE), sd = ~ sd(.x, na.rm=TRUE), median = ~ median(.x, na.rm=TRUE)))))

msg("Conteos principales (categorías):")
for (cname in intersect(c("popularity_label","artist_top20","title_has_feat","explicit","genre_primary"), names(df))) {
  msg("-> ", cname)
  print(df %>% count(.data[[cname]]) %>% arrange(desc(n)) %>% head(10))
}

# ------------------------------
# H) Visualizaciones (cada bloque con comentario objetivo y uso)
# ------------------------------
# Las gráficas se guardan en params$out_dir. Cada bloque tiene un comentario explicativo
# 1) Histograma de popularity
# Objetivo: mostrar la distribución de la variable objetivo (track_popularity).
# Uso: entender si es sesgada, identificar umbrales para clasificación o re-muestreo.
if ("track_popularity" %in% names(df)) {
  p_pop <- ggplot(df %>% filter(!is.na(track_popularity)), aes(x = track_popularity)) +
    geom_histogram(binwidth = 2, fill = "#2b8cbe", color = "white") +
    labs(title = "Distribución de Track Popularity",
         subtitle = "Objetivo: visualizar la distribución del target; útil para decidir bins/estrategia de muestreo",
         x = "Track popularity", y = "Conteo") +
    theme_minimal()
  ggsave(file.path(params$out_dir, "01_popularity_hist.png"), p_pop, width = 8, height = 5)
  msg("Guardado: 01_popularity_hist.png")
}

# 2) Top 20 artistas por número de tracks (barra)
# Objetivo: detectar artistas con alta representación (posible sesgo).
# Uso: decidir estrategias de reducción de cardinalidad o sampleo estratificado por artista.
if ("artist_name" %in% names(df)) {
  p_art <- df %>% count(artist_name, sort = TRUE) %>% slice_max(n, n = 20) %>%
    mutate(artist_name = fct_reorder(artist_name, n)) %>%
    ggplot(aes(x = artist_name, y = n)) + geom_col(fill = "#f768a1") + coord_flip() +
    labs(title = "Top 20 artistas por número de tracks",
         subtitle = "Objetivo: identificar artistas sobrerrepresentados (sesgo de artista).",
         x = "Artista", y = "Tracks") + theme_minimal()
  ggsave(file.path(params$out_dir, "02_top20_artists.png"), p_art, width = 10, height = 6)
  msg("Guardado: 02_top20_artists.png")
}

# 3) Popularidad media por año (línea)
# Objetivo: analizar tendencias temporales de popularity.
# Uso: detectar cambios en el tiempo, drift temporal y periodos relevantes para features temporales.
if ("release_year" %in% names(df) && any(!is.na(df$release_year))) {
  p_year <- df %>% filter(!is.na(release_year)) %>%
    group_by(release_year) %>% summarise(pop_mean = mean(track_popularity, na.rm = TRUE), n = n()) %>%
    ggplot(aes(x = release_year, y = pop_mean)) +
    geom_line(color = "#2ca25f") + geom_point() +
    scale_x_continuous(limits = params$year_limits, breaks = seq(params$year_limits[1], params$year_limits[2], by = 1)) +
    labs(title = "Popularidad media por año de lanzamiento (2000-2025)",
         subtitle = "Objetivo: ver la tendencia temporal en el rango 2000-2025; útil para detectar drift temporal",
         x = "Año", y = "Popularidad media") + theme_minimal() +
    theme(axis.text.x = element_text(angle = 60, hjust = 1))
  ggsave(file.path(params$out_dir, "03_popularity_by_year_2000_2025.png"), p_year, width = 12, height = 5)
  msg("Guardado: 03_popularity_by_year_2000_2025.png")
}

# 4) Número de tracks por año (barra)
# Objetivo: ver volumen de datos por año dentro del rango de interés.
# Uso: detectar años con pocos datos (posible filtrado) y decidir ponderaciones por año.
if ("release_year" %in% names(df)) {
  p_counts <- df %>% filter(!is.na(release_year)) %>% count(release_year) %>%
    ggplot(aes(x = release_year, y = n)) + geom_col(fill = "#8da0cb") +
    scale_x_continuous(limits = params$year_limits, breaks = seq(params$year_limits[1], params$year_limits[2], by = 1)) +
    labs(title = "Número de tracks por año (2000-2025)",
         subtitle = "Objetivo: mostrar volumen anual en 2000-2025; útil para evaluar representatividad temporal",
         x = "Año", y = "Número de tracks") + theme_minimal() +
    theme(axis.text.x = element_text(angle = 60, hjust = 1))
  ggsave(file.path(params$out_dir, "04_tracks_per_year_2000_2025.png"), p_counts, width = 12, height = 5)
  msg("Guardado: 04_tracks_per_year_2000_2025.png")
}

# 5) Distribución de duración (minutos)
# Objetivo: identificar outliers en duración y entender si es relevante para el target.
# Uso: feature selection (duración), clipping o normalización.
if ("duration_min" %in% names(df)) {
  p_dur <- df %>% filter(!is.na(duration_min)) %>%
    ggplot(aes(x = duration_min)) + geom_histogram(binwidth = 0.5, fill = "#8da0cb", color = "white") +
    labs(title = "Distribución de duración de las pistas (min)",
         subtitle = "Objetivo: detectar outliers y la forma general de la duración (útil para normalizar o clippear).",
         x = "Duración (min)", y = "Conteo") + theme_minimal()
  ggsave(file.path(params$out_dir, "05_duration_hist.png"), p_dur, width = 8, height = 5)
  msg("Guardado: 05_duration_hist.png")
}

# 6) Correlación entre variables numéricas (heatmap)
# Objetivo: detectar correlaciones fuertes y multicolinealidad; exportar CSV con valores
num_cols_all <- df %>% select(where(is.numeric)) %>% names()
if (length(num_cols_all) >= 2) {
if (length(num_cols_all) > params$max_vars_for_corr) {
var_dt <- df %>% select(all_of(num_cols_all)) %>% summarise(across(everything(), ~ var(.x, na.rm = TRUE))) %>% pivot_longer(everything(), names_to = "var", values_to = "v") %>% arrange(desc(v))
num_cols <- var_dt$var[1:params$max_vars_for_corr]
message("Seleccionadas top ", params$max_vars_for_corr, " variables por varianza para correlación: ", paste(num_cols, collapse = ", "))
} else { num_cols <- num_cols_all }


cor_mat <- df %>% select(all_of(num_cols)) %>% mutate(across(everything(), as.numeric)) %>% cor(use = "pairwise.complete.obs")
# exportar matriz
readr::write_csv(as.data.frame(cor_mat) %>% tibble::rownames_to_column("variable"), file.path(params$out_dir, "correlation_matrix.csv"))


# parámetros adaptativos
nvars <- length(num_cols)
plot_w <- ifelse(nvars <= 8, 6, ifelse(nvars <= 15, 9, 12))
plot_h <- plot_w
lab_size <- ifelse(nvars <= 8, 6, ifelse(nvars <= 15, 4, 3))


p_corr <- tryCatch({
ggcorrplot::ggcorrplot(cor_mat, method = "square", type = "lower", lab = TRUE, lab_size = lab_size, digits = 2, tl.cex = ifelse(nvars <= 12, 10, 8), tl.srt = 45, hc.order = TRUE, outline.color = "gray80", colors = c("#3b4cc0","white","#b2182b")) + ggtitle("Mapa de correlación (mejorado)")
}, error = function(e) {
message("ggcorrplot falló: ", e$message, " — usando fallback ggplot2")
NULL
})


if (is.null(p_corr)) {
cor_df_long <- as.data.frame(cor_mat) %>% tibble::rownames_to_column("row") %>% pivot_longer(-row, names_to = "col", values_to = "corr")
hc <- hclust(dist(1 - cor_mat))
ord <- hc$labels[hc$order]
cor_df_long$row <- factor(cor_df_long$row, levels = ord)
cor_df_long$col <- factor(cor_df_long$col, levels = ord)
p_corr <- ggplot(cor_df_long, aes(x = col, y = row, fill = corr)) + geom_tile(color = "white") + scale_fill_gradient2(low = "#3b4cc0", mid = "white", high = "#b2182b", limits = c(-1,1)) + geom_text(aes(label = sprintf("%.2f", corr)), size = lab_size) + theme_minimal() + theme(axis.text.x = element_text(angle = 60, hjust = 1)) + labs(title = "Mapa de correlación (fallback)")
}


ggsave(file.path(params$out_dir, "06_correlation_heatmap_improved.png"), p_corr, width = plot_w, height = plot_h, dpi = 300)
message("Correlación guardada: ", file.path(params$out_dir, "06_correlation_heatmap_improved.png"))
}

# 7) Energy vs Danceability (scatter) — coloreado por popularity
# Objetivo: explorar clusters de audio-features y su relación con popularity.
# Uso: posible agrupamiento, features composites o reglas basadas en audio-features.
if (all(c("energy","danceability","track_popularity") %in% names(df))) {
  p_ed <- df %>% filter(!is.na(energy) & !is.na(danceability)) %>%
    ggplot(aes(x = danceability, y = energy, color = track_popularity)) + geom_point(alpha = 0.6) +
    scale_color_viridis_c(option = "C") +
    labs(title = "Danceability vs Energy (color = popularity)",
         subtitle = "Objetivo: detectar relaciones entre audio-features y popularity; útil para features compuestas.",
         x = "Danceability", y = "Energy") + theme_minimal()
  ggsave(file.path(params$out_dir, "07_energy_dance_popularity.png"), p_ed, width = 8, height = 6)
  msg("Guardado: 07_energy_dance_popularity.png")
}

# 8) Boxplot de popularity por categoría (popularity_label)
# Objetivo: comprobar que los bins/labels reflejan diferencias en la variable target.
# Uso: validar thresholds para clasificación y balance de clases.
if (all(c("popularity_label","track_popularity") %in% names(df))) {
  p_box <- df %>% ggplot(aes(x = popularity_label, y = track_popularity, fill = popularity_label)) + geom_boxplot() +
    labs(title = "Popularity por categoría (bins)",
         subtitle = "Objetivo: mostrar diferencias entre etiquetas; útil para validación de bins.",
         x = "Etiqueta de popularity", y = "Track popularity") + theme_minimal() + theme(legend.position = "none")
  ggsave(file.path(params$out_dir, "08_popularity_by_label_boxplot.png"), p_box, width = 7, height = 5)
  msg("Guardado: 08_popularity_by_label_boxplot.png")
}

# ------------------------------
# I) Guardado de resumenes útiles (solo CSV procesado y resumen corto)
# ------------------------------
# No guardamos missing_summary, missingness_vars ni numeric_summary (según tu pedido)
readr::write_csv(df, params$processed_path)
msg("Archivo procesado guardado: ", params$processed_path)

# ------------------------------
# J) Recomendaciones (salida por consola)
# ------------------------------
cat("\\nRECOMENDACIONES MODELADO (resumen):\\n")
cat("- Imputar numéricos: median para skewed, mean para simétricos o KNN si hay dependencia espacial/temporal.\\n")
cat("- Escalar numéricos (z-score) o usar transformaciones log para followers/duración.\\n")
cat("- Reducir cardinalidad en artist_name/genre_primary (top-k o target-encoding).\\n")
cat("- Probar modelos de clasificación para popularity_label y regresión para track_popularity.\\n")
cat("- Evaluar VIF y eliminar multicolinealidad (usar correlation heatmap).\\n")
cat("- Considerar embeddings para artistas/géneros si se dispone de datos externos.\\n")

# ------------------------------
# K) Fin
# ------------------------------
msg("EDA y FE finalizado. Revisa carpeta: ", normalizePath(params$out_dir), " y archivo: ", params$processed_path)


# A tibble: 1 × 60
  track_number_mean track_number_sd track_number_median track_popularity_mean
              <dbl>           <dbl>               <dbl>                 <dbl>
1              5.75            6.02                   4                  52.2
# ℹ 56 more variables: track_popularity_sd <dbl>,
#   track_popularity_median <dbl>, track_duration_ms_mean <dbl>,
#   track_duration_ms_sd <dbl>, track_duration_ms_median <dbl>,
#   artist_popularity_mean <dbl>, artist_popularity_sd <dbl>,
#   artist_popularity_median <dbl>, artist_followers_mean <dbl>,
#   artist_followers_sd <dbl>, artist_followers_median <dbl>,
#   album_total_tracks_mean <dbl>, album_total_tracks_sd <dbl>, …
# A tibble: 4 × 2
  popularity_label     n
  <fct>            <int>
1 regular           4627
2 popular           3405
3 hit                697
4 mega_hit            49
# A tibble: 10 × 2
   artist_top20          n
   <fct>             <int>
 1 other              6978
 2 Taylor Swift        330
 3 The Weeknd     

Leyendo CSV: /content/track_data_final.csv
Lectura completada. Filas: 8778 Columnas: 15
Columnas detectadas: track_id, track_name, track_number, track_popularity, track_duration_ms, explicit, artist_name, artist_popularity, artist_followers, artist_genres, album_id, album_name, album_release_date, album_total_tracks, album_type
Aplicando feature engineering...
Procesado guardado en: processed_spotify_eda_final.csv
Resumen rápido (numéricas):
Conteos principales (categorías):
-> popularity_label
-> artist_top20
-> title_has_feat
-> explicit
-> genre_primary
Guardado: 01_popularity_hist.png
Guardado: 02_top20_artists.png
Guardado: 03_popularity_by_year_2000_2025.png
Guardado: 04_tracks_per_year_2000_2025.png
Guardado: 05_duration_hist.png
Correlación guardada: eda_plots/06_correlation_heatmap_improved.png
Guardado: 08_popularity_by_label_boxplot.png
Archivo procesado guardado: processed_spotify_eda_final.csv
EDA finalizado. Revisa carpeta: /content/eda_plots y archivo: processed_spotify_